# Table of Contents

1. <a href="#Gathered-Survey-Data">Gathered Survey Data</a>  
    1a. <a href="#Records">Records</a></n>  
    1b. <a href="#Reports">Reports</a></n>  
    1c. <a href="#Files---optional-attachments-to-individual-records">Files - optional attachments to individual records</a></n>  
2. <a href="#Survey-Metadata">Survey Metadata</a></n>  
    2a. <a href="#Metadata-(generic)">Metadata (generic)</a></n>  
    2b. <a href="#Field-Names">Field Names</a></n>  
    2c. <a href="#Forms/Instruments">Forms/Instruments</a></n>  
    2d. <a href="#Instrument-Event-Map">Instrument Event Map</a></n>  
3. <a href="#Survey-Settings-and-Other-Information">Survey Settings and Other Information</a></n>  
    3a. <a href="#Users">Users</a></n>  
    3b. <a href="#User-Roles">User Roles</a></n>  
    3c. <a href="#Data-Access-Groups-(DAGs)">Data Access Groups (DAGs)</a></n>  
    3d. <a href="#Logging">Logging</a></n>  
4. <a href="#Appendix">Appendix</a></n> <br>
    4a. <a href="#Example:-Uploading-Records-from-a-CSV">Example: Uploading Records from a CSV</a></n> <br>
    4b. <a href="#Import-Using-Native-API">Import Using Native API</a></n>

In [1]:
import json
import csv
import numpy as np
import requests
import pandas as pd

In [2]:
from redcap import Project

In [3]:
# Load API tokens from the json file
key = json.load(open('json_api_data.json'))
token = key['dev_token']['276']
url = key['dev_url']

In [4]:
# Set up PyCap project
project = Project(url, token) # Caitlin's project

In [5]:
# This is the heart of PyCap - This object is acted upon for all the actions we can take
project?

Type:           Project
String form:    <redcap.project.Project object at 0x00000260C5947FA0>
File:           c:\users\ekp0303\miniconda3\envs\transform22\lib\site-packages\redcap\project.py
Docstring:     
Main class for interacting with REDCap projects

Attributes:
    verify_ssl: Verify SSL, default True. Can pass path to CA_BUNDLE

Note:
    Your REDCap token should be kept **secret**! Treat it like a password
    and NEVER save it directly in your script/application. Rather it should be obscured
    and retrieved 'behind the scenes'. For example, saving the token as an environment
    variable and retrieving it with `os.getenv`. The creation of the `TOKEN` string in
    the example is not shown, for the above reasons

Examples:
    >>> from redcap import Project
    >>> URL = "https://redcapdemo.vanderbilt.edu/api/"
    >>> proj = Project(URL, TOKEN)
    >>> proj.field_names
    ['record_id', 'field_1', 'checkbox_field', 'upload_field']
    >>> proj.is_longitudinal
    True
    >>

# Gathered Survey Data

## Records

Records can be imported into a REDCap project using `import_records`.

If the record_id being imported already exists in the REDCap project, the imported data will overwrite the previously existing data for that record. Using a record_id that does not already exist will create a new record. The `force_auto_number = 'True'` arguement will automatically rename already exisitng record_ids to new record_ids during import. Unless you set the argument `overwrite = 'overwrite'`, exisitng data will not be overwritten by null values. See the <a href="#Note-on-overwrite-argument">Note on overwrite argument</a> and the [API Reference](http://redcap-tools.github.io/PyCap/api_reference/records/#redcap.methods.records.Records.import_records) for more infromation. 

In [6]:
# Define data to import
data2 = '[{"record_id": "3", "redcap_event_name": "personal_info_arm_1", "redcap_repeat_instrument": "", "redcap_repeat_instance": "","redcap_survey_identifier": "", "demographics_timestamp": "" ,"first_name": "John","last_name": "Doe","phone_num": "(999) 999-9999","zip_code": "98105"}]'

In [7]:
# json reads this in - the format is valid.
python_data = json.loads(data2)

In [8]:
# passing it as a string shows it flipped the quotes because python prefers single quotes
str(python_data)

"[{'record_id': '3', 'redcap_event_name': 'personal_info_arm_1', 'redcap_repeat_instrument': '', 'redcap_repeat_instance': '', 'redcap_survey_identifier': '', 'demographics_timestamp': '', 'first_name': 'John', 'last_name': 'Doe', 'phone_num': '(999) 999-9999', 'zip_code': '98105'}]"

PyCap `import_records` can use the json object we created, even with the quotes flipped.

In [9]:
project.import_records(python_data, force_auto_number = 'True')

{'count': 1}

In [10]:
# Here's the full function documentation
project.import_records?

Signature:
project.import_records(
    to_import: Union[str, List[Dict[str, Any]], ForwardRef('pd.DataFrame')],
    return_format_type: Literal['json', 'csv', 'xml'] = 'json',
    return_content: Literal['count', 'ids', 'auto_ids', 'nothing'] = 'count',
    overwrite: Literal['normal', 'overwrite'] = 'normal',
    import_format: Literal['json', 'csv', 'xml', 'df'] = 'json',
    date_format: Literal['YMD', 'DMY', 'MDY'] = 'YMD',
    force_auto_number: bool = False,
)
Docstring:
Import data into the REDCap Project

Args:
    to_import:
        Note:
            If you pass a df, csv, or xml string, you should use the
            `import_format` parameter appropriately.
        Note:
            Keys of the dictionaries should be subset of project's,
            fields, but this isn't a requirement. If you provide keys
            that aren't defined fields, the returned response will
            contain an `'error'` key.
    return_format_type:
        Response format. By default, respon

## Reports

N/A. Cannot be imported. 

## Files - optional attachments to individual records

In [11]:
import tempfile

In [12]:
tmp_file = tempfile.TemporaryFile()
project.import_file(record="10",
                 field="test_upload", 
                 file_name="../files/test_file.png",
                 file_object=tmp_file,
                 event="case_intake_arm_1")

[{}]

# Survey Metadata

## Metadata (generic)

In this example, we will export and import the same metadata so that now changes are actually made to the project.

In [13]:
metadata = project.metadata
metadata[0]

{'field_name': 'record_id',
 'form_name': 'demographics',
 'section_header': '',
 'field_type': 'text',
 'field_label': 'Study ID',
 'select_choices_or_calculations': '',
 'field_note': '',
 'text_validation_type_or_show_slider_number': '',
 'text_validation_min': '',
 'text_validation_max': '',
 'identifier': '',
 'branching_logic': '',
 'required_field': '',
 'custom_alignment': '',
 'question_number': '',
 'matrix_group_name': '',
 'matrix_ranking': '',
 'field_annotation': ''}

In [14]:
project.import_metadata(to_import=metadata)

30

## Field Names

N/A. Importing this would be covered by the metadata function.  
Can be exported using `project.field_names()`.

## Forms/Instruments

N/A. Importing this would be covered by the metadata function.  
Can be exported using `project.forms`.

## Instrument Event Map

Note that instrument event mapping is only applicable for longitudinal projects. You can run `project.is_longitudinal` to see if you have a longitudinal project or not. 

In [15]:
project.is_longitudinal

True

In [16]:
project.export_instrument_event_mappings()

[{'arm_num': 1,
  'unique_event_name': 'personal_info_arm_1',
  'form': 'demographics'},
 {'arm_num': 1, 'unique_event_name': 'case_intake_arm_1', 'form': 'symptoms'},
 {'arm_num': 1,
  'unique_event_name': 'case_intake_arm_1',
  'form': 'test_information'},
 {'arm_num': 1,
  'unique_event_name': 'notifications_arm_1',
  'form': 'close_contacts'},
 {'arm_num': 1,
  'unique_event_name': 'notifications_arm_1',
  'form': 'work_information'}]

In this example, we will import the same event mapping schema that we exported so that no changes are actually made to the project.

In [17]:
inst_map = [{'arm_num': '1', 'unique_event_name': 'personal_info_arm_1', 'form': 'demographics'},{'arm_num': '1', 'unique_event_name': 'case_intake_arm_1', 'form': 'symptoms'},{'arm_num': '1','unique_event_name': 'case_intake_arm_1','form': 'test_information'},{'arm_num': '1','unique_event_name': 'notifications_arm_1','form': 'close_contacts'},{'arm_num': '1','unique_event_name': 'notifications_arm_1','form': 'work_information'}]

Note: `import_instrument_event_mappings` is only available in the latest version of Pycap (2.6.0) imported from GitHub

In [18]:
# This is not wokring 
project.import_instrument_event_mappings(inst_map)

5

### Specifying Repeat Instruments

In [19]:
rep_instruments = project.export_repeating_instruments_events()
pd.DataFrame.from_dict(rep_instruments)

,event_name,form_name,custom_form_label
0,case_intake_arm_1,,
1,notifications_arm_1,close_contacts,


It's easier to understand when viewed as a dataframe. The `form_name` variable is blank in row 0 because the the entire 'case_intake_arm_1' event is repeated, whereas, only the 'close_contacts' form is repeated within the 'notifications_arm_1' event. 

In [20]:
rep_instruments

[{'event_name': 'case_intake_arm_1', 'form_name': '', 'custom_form_label': ''},
 {'event_name': 'notifications_arm_1',
  'form_name': 'close_contacts',
  'custom_form_label': ''}]

In [21]:
rep_instruments_new = [{'event_name': 'case_intake_arm_1', 'form_name': '', 'custom_form_label': ''},
 {'event_name': 'notifications_arm_1',
  'form_name': '',
  'custom_form_label': ''}]

In [22]:
project.import_repeating_instruments_events(rep_instruments_new)

2

Because the `form_name` value was removed from the 'notifications_arm_1' event, the entire event will now repeat. We can change it back in the next line. 

In [23]:
project.import_repeating_instruments_events(rep_instruments)

2

# Survey Settings and Other Information

## Users

In [24]:
# View current users in the REDCap project
project.export_users(format_type = 'df')

,username,email,firstname,lastname,expiration,data_access_group,data_access_group_id,design,alerts,user_rights,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_all_forms,lock_records,lock_records_customization,forms,forms_export
0,alexey.gilman@doh.wa.gov,Alexey.Gilman@doh.wa.gov,Alexey,Gilman,NaN,full_access,2029.0,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
1,caitlin.drover@doh.wa.gov,Caitlin.Drover@doh.wa.gov,Caitlin,Drover,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
2,emily.pearman@doh.wa.gov,emily.pearman@doh.wa.gov,Emily,Pearman,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."


In [25]:
# the most basic, add a new user
new_user = [{'username':'bambergertestemail@gmail.com'}]
project.import_users(new_user)

1

In [26]:
# You can see the new user has been added
project.export_users(format_type = 'df')

,username,email,firstname,lastname,expiration,data_access_group,data_access_group_id,design,alerts,user_rights,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_all_forms,lock_records,lock_records_customization,forms,forms_export
0,alexey.gilman@doh.wa.gov,Alexey.Gilman@doh.wa.gov,Alexey,Gilman,NaN,full_access,2029.0,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
1,bambergertestemail@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
2,caitlin.drover@doh.wa.gov,Caitlin.Drover@doh.wa.gov,Caitlin,Drover,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
3,emily.pearman@doh.wa.gov,emily.pearman@doh.wa.gov,Emily,Pearman,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."


In [73]:
# View how the data is formated as a json
project.export_users()

[{'username': 'alexey.gilman@doh.wa.gov',
  'email': 'Alexey.Gilman@doh.wa.gov',
  'firstname': 'Alexey',
  'lastname': 'Gilman',
  'expiration': '',
  'data_access_group': 'full_access',
  'data_access_group_id': 2029,
  'design': 1,
  'alerts': 1,
  'user_rights': 1,
  'data_access_groups': 1,
  'reports': 1,
  'stats_and_charts': 1,
  'manage_survey_participants': 1,
  'calendar': 1,
  'data_import_tool': 1,
  'data_comparison_tool': 1,
  'logging': 1,
  'file_repository': 1,
  'data_quality_create': 1,
  'data_quality_execute': 1,
  'api_export': 1,
  'api_import': 1,
  'api_modules': 0,
  'mobile_app': 1,
  'mobile_app_download_data': 0,
  'record_create': 1,
  'record_rename': 1,
  'record_delete': 1,
  'lock_records_all_forms': 0,
  'lock_records': 0,
  'lock_records_customization': 1,
  'forms': {'demographics': 3,
   'symptoms': 1,
   'test_information': 1,
   'close_contacts': 1,
   'work_information': 1},
  'forms_export': {'demographics': 1,
   'symptoms': 1,
   'test_infor

When specifying form permissions, you need to write them as their own dictionary within the dictionary. See the example below. Refer to the [documentation](http://redcap-tools.github.io/PyCap/api_reference/users/#redcap.methods.users.Users.import_users) for a list of all user rights options that can be included. 

In [27]:
# More advanced
# Fields are mostly 1s or 0s, with 1 granting permission, but you can also set data access group in here with name or id
advanced_user = [{'username':'bambergertestemail@gmail.com', 'reports':1, 'api_export':1, 'record_rename':1, 'forms': {'demographics': 3,
  'symptoms': 1,
  'test_information': 1,
  'close_contacts': 1,
  'work_information': 1}}]
project.import_users(advanced_user)

1

In [28]:
# You can see the user now has the specified permissions
project.export_users(format_type = 'df')

,username,email,firstname,lastname,expiration,data_access_group,data_access_group_id,design,alerts,user_rights,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_all_forms,lock_records,lock_records_customization,forms,forms_export
0,alexey.gilman@doh.wa.gov,Alexey.Gilman@doh.wa.gov,Alexey,Gilman,NaN,full_access,2029.0,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
1,bambergertestemail@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,1,0,0,0,0,"demographics:3,symptoms:1,test_information:1,c...","demographics:0,symptoms:0,test_information:0,c..."
2,caitlin.drover@doh.wa.gov,Caitlin.Drover@doh.wa.gov,Caitlin,Drover,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."
3,emily.pearman@doh.wa.gov,emily.pearman@doh.wa.gov,Emily,Pearman,NaN,NaN,NaN,1,1,1,...,1,0,1,1,1,0,0,1,"demographics:3,symptoms:1,test_information:1,c...","demographics:1,symptoms:1,test_information:1,c..."


You can also delete users. 

In [29]:
project.delete_users(['bambergertestemail@gmail.com'])

1

## User Roles

You can add user roles and add user to these defined user roles.

In [30]:
# Export and view user roles currently in project
project.export_user_roles(format_type = 'df')
# everything is a 0 or 1 to denote true/false except for unique_role_name and role_label

,unique_role_name,role_label,design,alerts,user_rights,data_access_groups,reports,stats_and_charts,manage_survey_participants,calendar,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_customization,lock_records,lock_records_all_forms,forms,forms_export
0,U-297XW9NYE4,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
1,U-428EKXLKWH,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
2,U-5354FA3HYL,Admin,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,0,"demographics:3,symptoms:1,test_information:1,c...","demographics:3,symptoms:1,test_information:1,c..."
3,U-638JK7XXLM,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
4,U-754CK8KWDJ,Advanced Role,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:3,test_information:0,c...","demographics:0,symptoms:3,test_information:0,c..."
5,U-762XYDFYFE,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
6,U-781PK3L9JL,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
7,U-896RPARYHM,Advanced Role,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:3,test_information:0,c...","demographics:0,symptoms:3,test_information:0,c..."


In [31]:
# Minimum command: all permissions default to 0 (no access) unless otherwise specified
new_role = [{'role_label':'Test Role'}]
project.import_user_roles(new_role)

1

Note forms permissions assignment need to be written as a dictionary within a dictionary

In [32]:
# more advanced
role_advanced = [{'role_label':'Advanced Role', 'api_export':'1', 'api_import':'1', 'logging':'1', 'forms': {'demographics': 0,
  'symptoms': 3,
  'test_information': 0,
  'close_contacts': 3,
  'work_information': 0}, 'alerts':'1'}]

project.import_user_roles(role_advanced)

1

In [55]:
# View the new role that was added
project.export_user_roles(format_type = 'df')

,unique_role_name,role_label,design,alerts,user_rights,data_access_groups,reports,stats_and_charts,manage_survey_participants,calendar,...,mobile_app,mobile_app_download_data,record_create,record_rename,record_delete,lock_records_customization,lock_records,lock_records_all_forms,forms,forms_export
0,U-297XW9NYE4,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
1,U-428EKXLKWH,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
2,U-5354FA3HYL,Admin,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,0,"demographics:3,symptoms:1,test_information:1,c...","demographics:3,symptoms:1,test_information:1,c..."
3,U-638JK7XXLM,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
4,U-754CK8KWDJ,Advanced Role,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:3,test_information:0,c...","demographics:0,symptoms:3,test_information:0,c..."
5,U-762XYDFYFE,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
6,U-781PK3L9JL,Test Role,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:0,test_information:0,c...","demographics:0,symptoms:0,test_information:0,c..."
7,U-896RPARYHM,Advanced Role,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"demographics:0,symptoms:3,test_information:0,c...","demographics:0,symptoms:3,test_information:0,c..."


### User Role Assignment

You can view and edit which users are assigned to which user role

In [33]:
# View current user role assignments
project.export_user_role_assignment()

[{'username': 'alexey.gilman@doh.wa.gov',
  'unique_role_name': 'U-5354FA3HYL',
  'data_access_group': 'full_access'},
 {'username': 'caitlin.drover@doh.wa.gov',
  'unique_role_name': 'U-5354FA3HYL',
  'data_access_group': ''},
 {'username': 'emily.pearman@doh.wa.gov',
  'unique_role_name': 'U-5354FA3HYL',
  'data_access_group': ''}]

Can use `import_user_role_assignemnt` to assign a user to a role. Must use the unique role names listed above when specifying the role you want to assign a user to.  

In [34]:
role_assigne = [{'username': 'alexey.gilman@doh.wa.gov', 'unique_role_name': 'U-896RPARYHM'}]

In [35]:
project.import_user_role_assignment(role_assigne)

1

In [36]:
# View new user role assignments
project.export_user_role_assignment()

[{'username': 'alexey.gilman@doh.wa.gov',
  'unique_role_name': 'U-896RPARYHM',
  'data_access_group': 'full_access'},
 {'username': 'caitlin.drover@doh.wa.gov',
  'unique_role_name': 'U-5354FA3HYL',
  'data_access_group': ''},
 {'username': 'emily.pearman@doh.wa.gov',
  'unique_role_name': 'U-5354FA3HYL',
  'data_access_group': ''}]

In [37]:
# Move user back to original role
role_assigne2 = [{'username': 'alexey.gilman@doh.wa.gov', 'unique_role_name': 'U-5354FA3HYL'}]
project.import_user_role_assignment(role_assigne2)

1

## Data Access Groups (DAGs)

In [38]:
# View the current DAGs and their ID's
project.export_dags(format_type = 'df')

,data_access_group_name,unique_group_name,data_access_group_id
0,Full Access,full_access,2029
1,Limited Access,limited_access,2028
2,Test DAG,test_dag,2030
3,Test DAG,test_dagb,2031


Create a new DAG

In [39]:
new_dag = [{'data_access_group_name':'Test DAG', 'unique_group_name':''}]
project.import_dags(new_dag)

1

In [75]:
# View the new DAG and their ID's
project.export_dags(format_type = 'df')

,data_access_group_name,unique_group_name,data_access_group_id
0,Full Access,full_access,2029
1,Limited Access,limited_access,2028
2,Test DAG,test_dag,2030
3,Test DAG,test_dagb,2031


Remember that you can assign a user to a specific DAG by including it in the User Roles import API. 

## Logging

N/A. Cannot be imported.  
Can be exported using `project.export_logging()`.

# Appendix

## Example: Uploading Records from a CSV

In this example, we have a csv named "data_to_import.csv", with new records to upload.

When reading a csv as a pandas dataframe, python will take any numeric column with missing data and convert them to [float with NaN inserted](https://stackoverflow.com/questions/39666308/pd-read-csv-by-default-treats-integers-like-floats) in the blank cells. In longitudinal projects, we expect lots of blank cells since data is wide and for each row, only columns relevant to that event/instrument are filled out. Many of REDCap's field types (checkbox, yes/no, radio, and form_complete variables) are integers. Pandas will convert these columns to float variables with a decimal place added (i.e. 1.0 instead of 1 for 'Yes' in a yes/no field) and import to these integer field types in REDCap will fail.

### Issues introduced by `pd.read_csv`

In [6]:
# Read and view data to import
df_to_import = pd.read_csv("../files/data_to_import.csv")
df_to_import.head()

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_survey_identifier,demographics_timestamp,first_name,last_name,phone_num,zip_code,...,cc_phone,cc_email,close_contacts_complete,supervisor_name,supervisor_email,work_inperson_yesno,work_date,work_contagious,work_contagious_calc,work_information_complete
0,3,personal_info_arm_1,NaN,NaN,NaN,NaN,John,Doe,(999) 999-9999,98105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,notifications_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Boss,NaN,0.0,NaN,0.0,NaN,2.0
2,3,case_intake_arm_1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,notifications_arm_1,close_contacts,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,fake_email@gmail.com,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,notifications_arm_1,close_contacts,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,(999) 999-9999,fake_email@gmail.com,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


See how the redcap_repeat_instance, close_contacts_complete, and work_inperson_yesno are some of the many fields that were converted to float with an added decimal. If I try to import this dataset it will produce many errors.

In [7]:
# specify `import_format` = df for a pandas data frame
project.import_records(df_to_import, import_format = 'df', force_auto_number=True, date_format = 'MDY')

RequestException: {'error': '"97","zip_code","98105.0","The value you provided could not be validated because it does not follow the expected format. Please try again."\n"97","ethnicity","1.0","The value is not a valid category for ethnicity"\n"97","race","4.0","The value is not a valid category for race"\n"97","gender","1.0","The value is not a valid category for gender"\n"97","demographics_complete","2.0","The value is not a valid category for demographics_complete"\n"97","work_inperson_yesno","0.0","The value is not a valid category for work_inperson_yesno"\n"97","work_contagious","0.0","The value is not a valid category for work_contagious"\n"97","work_information_complete","2.0","The value is not a valid category for work_information_complete"\n"97","redcap_repeat_instance","1.0","(redcap_event_name=""case_intake_arm_1"") redcap_repeat_instance must have a numeric value between \'1\' and \'32767\' for any Repeating Event or Repeating Instrument.  Please add a valid numeric value t

### Solution - Convert All Floats to Int64 Pandas Datatype

Int64 is a unique pandas datatype that allows numeric fields to contain missing values. For more information read the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.Int64Dtype.html)  

Note: Before applying this solution, ensure that there are no numeric fields in your REDCap Project that should have decimals (you will not want to convert these variables to int64 since they would lose their decimal places). Make sure you are familiar with your project's metadata. All radio, checkboxes, yes/no, redcap_repeat_instance, and form_complete variables need to be integers. In REDCap, actual numeric fields are stored as text fields with optional validation. Any text field in REDCap with no validation or with 'numeric' as their validation type, would allow numbers with decimal places. Any text fields with other validations types (i.e. zip code, phone number, integer) will not allow import of decimal places. 

In [79]:
float_list = df_to_import.select_dtypes(include=[np.float64]).columns.values.tolist()
print(float_list)

['redcap_repeat_instance', 'redcap_survey_identifier', 'demographics_timestamp', 'zip_code', 'age', 'ethnicity', 'race', 'gender', 'demographics_complete', 'symptoms_yesno', 'symptoms_exp___1', 'symptoms_exp___2', 'symptoms_exp___3', 'symptoms_exp___4', 'symptoms_exp___5', 'symptoms_exp___6', 'symptoms_exp___7', 'symptoms_exp___8', 'symptoms_exp___9', 'symptoms_exp___10', 'symptoms_exp___11', 'symptom_notes', 'symptoms_complete', 'test_yesno', 'test_positive_yesno', 'prior_covid_yesno', 'test_information_complete', 'close_contacts_complete', 'work_inperson_yesno', 'work_contagious', 'work_contagious_calc', 'work_information_complete']


At this point, if needed, you can remove any variables from this list that you need to keep in float format. 

In [65]:
df_to_import[float_list] = df_to_import[float_list].apply(lambda x: x.astype("Int64"))

In [66]:
df_to_import.head()

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_survey_identifier,demographics_timestamp,first_name,last_name,phone_num,zip_code,...,cc_phone,cc_email,close_contacts_complete,supervisor_name,supervisor_email,work_inperson_yesno,work_date,work_contagious,work_contagious_calc,work_information_complete
0,3,personal_info_arm_1,NaN,<NA>,<NA>,<NA>,John,Doe,(999) 999-9999,98105,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>
1,3,notifications_arm_1,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,NaN,NaN,<NA>,Boss,NaN,0,NaN,0,<NA>,2
2,3,case_intake_arm_1,NaN,1,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>
3,3,notifications_arm_1,close_contacts,1,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,(999) 999-9999,fake_email@gmail.com,2,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>
4,3,notifications_arm_1,close_contacts,2,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,(999) 999-9999,fake_email@gmail.com,2,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>


You can now see the `redcap_repeat_instance` and `clost_contacts_complete` are in integer format. The <NA> seen in the blank cells will not interfere with data import. Now you can make any edits necessary including numeric specific transformations on your integer fields. 

In [72]:
# In this example lets quickly edit a name 
df_to_import.loc[df_to_import.first_name == 'John', 'first_name'] = 'Jon'

In [73]:
df_to_import.head()

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_survey_identifier,demographics_timestamp,first_name,last_name,phone_num,zip_code,...,cc_phone,cc_email,close_contacts_complete,supervisor_name,supervisor_email,work_inperson_yesno,work_date,work_contagious,work_contagious_calc,work_information_complete
0,3,personal_info_arm_1,NaN,<NA>,<NA>,<NA>,Jon,Doe,(999) 999-9999,98105,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>
1,3,notifications_arm_1,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,NaN,NaN,<NA>,Boss,NaN,0,NaN,0,<NA>,2
2,3,case_intake_arm_1,NaN,1,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>
3,3,notifications_arm_1,close_contacts,1,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,(999) 999-9999,fake_email@gmail.com,2,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>
4,3,notifications_arm_1,close_contacts,2,<NA>,<NA>,NaN,NaN,NaN,<NA>,...,(999) 999-9999,fake_email@gmail.com,2,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>


In [74]:
# Import data
project.import_records(df_to_import, force_auto_number=True, date_format = 'MDY', import_format = 'df')

{'count': 4}

### Note on importing dates

Date fields in REDCap are character fields with a designated date validation added. There are many different types of date validations/formats that can be chosen for a date field. All date fields must be imported to REDCap only in the Y-M-D format, regardless of the specific date format designated for this field in the REDCap project. Fortunately, Pycap `import_records` will automatically convert dates to the proper format as long as the input format of your dates are specified under the `date_format` argument. If you have different date fields in different formats you will need to first get them in all the same format for this argument to work.  

### Note on `force auto_number`

Another unique feature of pycap `import_records` is the `force_auto_number` argument which, if set to 'True' it will automatically reasign your record id's to new record ids that will import to REDCap as new recrods. If set to 'False' and your record id's to import already exist in REDCap, they will overwrite the existing REDCap records during import. 

### Note on `overwrite` argument

The overwrite argument is set to 'normal' as default, under this setting, if blank values are imported for fields on existing REDCap records and that data is not missing in REDCap, these values will not be overwritten as missing. If you want to overwrite existing data as missing, be sure to include `overwrite = 'overwrite'` as an argument in `import_records` 

## Import Using Native API

### Records: Native API

In [27]:
data2 = '[{"record_id": "3", "redcap_event_name": "personal_info_arm_1", "redcap_repeat_instrument": "", "redcap_repeat_instance": "","redcap_survey_identifier": "", "demographics_timestamp": "" ,"first_name": "John","last_name": "Doe","phone_num": "(999) 999-9999","zip_code": "98105"}]'

- For the native API, must format data as '[{"key":"value"}]'
- [{'key':'value'}]" doesn't work - that's not valid json.
- Annoying, since python env defaults to ' over " but it's only a factor for these few methods.

In [32]:
# native API method
#!/usr/bin/env python
import requests
data = {
    'token': longitudinal_token,
    'content': 'record',
    'action': 'import',
    'format': 'json',
    'type': 'flat',
    'overwriteBehavior': 'normal',
    'forceAutoNumber': 'false',
    'data': data2,
    'returnContent': 'count',
    'returnFormat': 'json'
}

r = requests.post('https://dev-redcap.doh.wa.gov/api/',data=data)
print('HTTP Status: ' + str(r.status_code))
print(r.json())

HTTP Status: 200
{'count': 1}


### Instrument Event Mapping: Native API

In [38]:
inst_map = '[{"arm_num": "1", "unique_event_name": "personal_info_arm_1", "form": "demographics"},{"arm_num": "1", "unique_event_name": "case_intake_arm_1", "form": "symptoms"},{"arm_num": "1","unique_event_name": "case_intake_arm_1","form": "test_information"}, {"arm_num": "1","unique_event_name": "notifications_arm_1","form": "close_contacts"}, {"arm_num": "1","unique_event_name": "notifications_arm_1", "form": "work_information"}]'

In [47]:
# Native API can import instrument event mapping.
data = {
    'token': token,
    'content': 'formEventMapping',
    'data': inst_map,
    'format': 'json',
    'returnFormat': 'json'
}
r = requests.post('https://dev-redcap.doh.wa.gov/api/',data=data)
print('HTTP Status: ' + str(r.status_code))
print(r.json())

HTTP Status: 200
5
